# add_refseq_genenames.ipynb
## This script takes in a refseq annotation bedfile and a set of geneID/transcript equivalencies and adds genenames to bedfile

### Import libraries

In [57]:
import math

import numpy as np
import pandas as pd

### Define refseq input and equivalency file

In [58]:
file1 = ["/home/zarko/Data/annotation/mm10/" "mm10_refseq.bed"]
file2 = ["/home/zarko/Data/annotation/mm10/" "transcript_headings.txt"]
basename = "mm10_refseq"

### Dump input files into variables

In [60]:
with open(file1[0]) as f:
    reg = []
    for line in f:
        entry = list(line.strip().split("\t"))
        reg.append(entry)

regions = pd.DataFrame(reg)

with open(file2[0]) as f:
    eq = []
    for line in f:
        transcript = line.strip().split(">")[1].split(" Mus")[0]
        geneID = line.strip().split("(")[-1].split(")")[0]
        eq.append([transcript, geneID])

equiv = pd.DataFrame(data=eq, columns=["transcript", "geneID"])

### Add gene names to new column of regions df

In [61]:
geneids_to_add = []
df_indeces = []
for i in range(len(regions)):
    if regions[3].iloc[i] in equiv["transcript"].values:
        df_indeces.append(i)
        to_add = equiv[equiv["transcript"] == regions[3].iloc[i]]["geneID"].iloc[0]
        geneids_to_add.append(to_add)

filt_regions = regions.loc[df_indeces]
filt_regions["12"] = geneids_to_add

### Make outfile names and write out data

In [62]:
indir = file1[0].split("/")
outdir = "/".join(indir[0:-1])
outfile = "".join([outdir, "/", basename, "_genenames_included.bed"])

with open(outfile, "wt") as f:
    for i in range(len(filt_regions)):
        entry = "\t".join(filt_regions.iloc[i])
        f.write("".join([entry, "\n"]))